In [ ]:
#@title ## 📊 Comparação de Acurácia em Testes de Perguntas e Respostas (vs Llama-2 7b)

#@markdown ### Instruções:
#@markdown 1. Execute esta célula.
#@markdown 2. Para cada um dos 5 benchmarks listados abaixo, insira o número de respostas corretas obtidas pelo modelo base (Llama-2 7b) e por cada um dos modelos de comparação.
#@markdown 3. O número total de questões para cada benchmark já está pré-definido.
#@markdown 4. O script calculará as acurácias observadas, a diferença entre elas, o intervalo de confiança de 95% para essa diferença e determinará a significância estatística.
#@markdown 5. Os resultados de todas as comparações serão compilados e exibidos em uma tabela ao final.
#@markdown 6. Esse script foi feito visando a execução no ambiente Colab.

# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
from google.colab import files # Para importar arquivos no Colab.
from scipy import stats # Para o z-score (stats.norm.ppf)
import math # Para sqrt
import csv

In [2]:
# --- Seção de Entrada de Dados ---

#@markdown ---
#@markdown ### 📝 Benchmark 1: TruthfulQA mc1 (Total: 817 questões)
#@markdown Número de respostas corretas para cada modelo neste benchmark:
#@markdown Llama-2 7b (Base):
respostas_corretas_base_tqa_mc1 = 120 #@param {type:"integer"}
#@markdown KCA:
respostas_corretas_kca_tqa_mc1 = 319 #@param {type:"integer"}
#@markdown ICD:
respostas_corretas_icd_tqa_mc1 = 145 #@param {type:"integer"}
#@markdown WikiChat:
respostas_corretas_wikichat_tqa_mc1 = 220 #@param {type:"integer"}
#@markdown RepE:
respostas_corretas_repe_tqa_mc1 = 140 #@param {type:"integer"}

#@markdown ---
#@markdown ### 📝 Benchmark 2: TruthfulQA mc2 (Total: 817 questões)
#@markdown Número de respostas corretas para cada modelo neste benchmark:
#@markdown Llama-2 7b (Base):
respostas_corretas_base_tqa_mc2 = 200 #@param {type:"integer"}
#@markdown KCA:
respostas_corretas_kca_tqa_mc2 = 427 #@param {type:"integer"}
#@markdown ICD:
respostas_corretas_icd_tqa_mc2 = 391 #@param {type:"integer"}
#@markdown WikiChat:
respostas_corretas_wikichat_tqa_mc2 = 433 #@param {type:"integer"}
#@markdown RepE:
respostas_corretas_repe_tqa_mc2 = 255 #@param {type:"integer"}

#@markdown ---
#@markdown ### 📝 Benchmark 3: ARC Easy (Total: 5197 questões)
#@markdown Número de respostas corretas para cada modelo neste benchmark:
#@markdown Llama-2 7b (Base):
respostas_corretas_base_arc_e = 1634 #@param {type:"integer"}
#@markdown KCA:
respostas_corretas_kca_arc_e = 3163 #@param {type:"integer"}
#@markdown ICD:
respostas_corretas_icd_arc_e = 2927 #@param {type:"integer"}
#@markdown WikiChat:
respostas_corretas_wikichat_arc_e = 3064 #@param {type:"integer"}
#@markdown RepE:
respostas_corretas_repe_arc_e = 1832 #@param {type:"integer"}

#@markdown ---
#@markdown ### 📝 Benchmark 4: ARC Challenge (Total: 2590 questões)
#@markdown Número de respostas corretas para cada modelo neste benchmark:
#@markdown Llama-2 7b (Base):
respostas_corretas_base_arc_c = 854 #@param {type:"integer"}
#@markdown KCA:
respostas_corretas_kca_arc_c = 1048 #@param {type:"integer"}
#@markdown ICD:
respostas_corretas_icd_arc_c = 1039 #@param {type:"integer"}
#@markdown WikiChat:
respostas_corretas_wikichat_arc_c = 1039 #@param {type:"integer"}
#@markdown RepE:
respostas_corretas_repe_arc_c = 822 #@param {type:"integer"}

#@markdown ---
#@markdown ### 📝 Benchmark 5: OAB (Total: 2210 questões)
#@markdown Número de respostas corretas para cada modelo neste benchmark:
#@markdown Llama-2 7b (Base):
respostas_corretas_base_oab = 478 #@param {type:"integer"}
#@markdown KCA:
respostas_corretas_kca_oab = 655 #@param {type:"integer"}
#@markdown ICD:
respostas_corretas_icd_oab = 558 #@param {type:"integer"}
#@markdown WikiChat:
respostas_corretas_wikichat_oab = 605 #@param {type:"integer"}
#@markdown RepE:
respostas_corretas_repe_oab = 524 #@param {type:"integer"}

In [ ]:
def calcular_estatisticas_comparacao_acuracia(respostas_corretas_M, N_M, respostas_corretas_B, N_B, alfa=0.05):
    """
    Calcula o IC de 95% para a diferença entre duas proporções independentes.

    Args = 
        respostas_corretas_M (int): Número de respostas corretas para o modelo de mitigação/comparação.
        N_M (int): Número total de questões para o modelo de mitigação/comparação.
        respostas_corretas_B (int): Número de respostas corretas para o modelo base.
        N_B (int): Número total de questões para o modelo base.
        alfa (float): Nível de significância (padrão 0.05 para IC de 95%).

    Returns =
        dict: Um dicionário contendo as estatísticas da comparação.
    """
    # Validação básica das entradas
    if N_M <= 0 or N_B <= 0:
        return {
            "acuracia_M": float('nan'), "acuracia_B": float('nan'),
            "diferenca_acuracias": float('nan'), "erro_padrao_diferenca": float('nan'),
            "ic_inferior": float('nan'), "ic_superior": float('nan'),
            "estatisticamente_significativo": "N/A",
            "interpretacao": "Número total de questões (N) não pode ser zero ou negativo."
        }
    if not (0 <= respostas_corretas_M <= N_M and 0 <= respostas_corretas_B <= N_B):
        return {
            "acuracia_M": float('nan'), "acuracia_B": float('nan'),
            "diferenca_acuracias": float('nan'), "erro_padrao_diferenca": float('nan'),
            "ic_inferior": float('nan'), "ic_superior": float('nan'),
            "estatisticamente_significativo": "N/A",
            "interpretacao": "Número de respostas corretas deve estar entre 0 e N."
        }

    # Calcula as acurácias observadas (proporções)
    acuracia_observada_M = respostas_corretas_M / N_M
    acuracia_observada_B = respostas_corretas_B / N_B

    diferenca_acuracias = acuracia_observada_M - acuracia_observada_B

    # Erro padrão da diferença entre duas proporções
    termo_M = (acuracia_observada_M * (1 - acuracia_observada_M)) / N_M
    termo_B = (acuracia_observada_B * (1 - acuracia_observada_B)) / N_B

    # Safeguard para termos negativos devido a problemas de ponto flutuante (improvável com validação)
    if termo_M < 0: termo_M = 0
    if termo_B < 0: termo_B = 0

    erro_padrao_diferenca = math.sqrt(termo_M + termo_B)

    # Valor crítico Z para o nível de confiança
    z_critico = stats.norm.ppf(1 - alfa / 2) # Para IC 95%, alfa=0.05, z_critico ~ 1.96

    margem_erro = z_critico * erro_padrao_diferenca

    ic_inferior = diferenca_acuracias - margem_erro
    ic_superior = diferenca_acuracias + margem_erro

    estatisticamente_significativo = not (ic_inferior <= 0 <= ic_superior)

    # Cria uma interpretação textual (útil tanto para debug quanto para a visualização mais rápida do resultado)
    interpretacao = f"A acurácia do modelo comparado foi "
    if estatisticamente_significativo:
        if diferenca_acuracias > 0:
            interpretacao += "estatisticamente SUPERIOR."
        else: # diferenca_acuracias < 0 (não pode ser 0 se significativo e IC não contém 0)
            interpretacao += "estatisticamente INFERIOR."
    else:
        interpretacao += "NÃO ESTATISTICAMENTE DIFERENTE da do modelo base."

    interpretacao += f" O IC de 95% para a diferença (p_M - p_B) é [{ic_inferior:.4f}, {ic_superior:.4f}]."
    if not estatisticamente_significativo:
         interpretacao += " (Contém zero)."
    else:
         interpretacao += " (Não contém zero)."


    return {
        "acuracia_M": acuracia_observada_M,
        "acuracia_B": acuracia_observada_B,
        "diferenca_acuracias": diferenca_acuracias,
        "erro_padrao_diferenca": erro_padrao_diferenca,
        "ic_inferior": ic_inferior,
        "ic_superior": ic_superior,
        "estatisticamente_significativo": estatisticamente_significativo,
        "interpretacao": interpretacao
    }

In [4]:
dados_benchmarks_input = [
    {
        "nome_benchmark": "TruthfulQA mc1", "total_questoes": 817,
        "respostas_base": respostas_corretas_base_tqa_mc1,
        "modelos_comparacao_input": [
            {"nome_modelo": "KCA", "respostas_corretas": respostas_corretas_kca_tqa_mc1},
            {"nome_modelo": "ICD", "respostas_corretas": respostas_corretas_icd_tqa_mc1},
            {"nome_modelo": "WikiChat", "respostas_corretas": respostas_corretas_wikichat_tqa_mc1},
            {"nome_modelo": "RepE", "respostas_corretas": respostas_corretas_repe_tqa_mc1},
        ]
    },
    {
        "nome_benchmark": "TruthfulQA mc2", "total_questoes": 817,
        "respostas_base": respostas_corretas_base_tqa_mc2,
        "modelos_comparacao_input": [
            {"nome_modelo": "KCA", "respostas_corretas": respostas_corretas_kca_tqa_mc2},
            {"nome_modelo": "ICD", "respostas_corretas": respostas_corretas_icd_tqa_mc2},
            {"nome_modelo": "WikiChat", "respostas_corretas": respostas_corretas_wikichat_tqa_mc2},
            {"nome_modelo": "RepE", "respostas_corretas": respostas_corretas_repe_tqa_mc2},
        ]
    },
    {
        "nome_benchmark": "ARC Easy", "total_questoes": 5197,
        "respostas_base": respostas_corretas_base_arc_e,
        "modelos_comparacao_input": [
            {"nome_modelo": "KCA", "respostas_corretas": respostas_corretas_kca_arc_e},
            {"nome_modelo": "ICD", "respostas_corretas": respostas_corretas_icd_arc_e},
            {"nome_modelo": "WikiChat", "respostas_corretas": respostas_corretas_wikichat_arc_e},
            {"nome_modelo": "RepE", "respostas_corretas": respostas_corretas_repe_arc_e},
        ]
    },
    {
        "nome_benchmark": "ARC Challenge", "total_questoes": 2590,
        "respostas_base": respostas_corretas_base_arc_c,
        "modelos_comparacao_input": [
            {"nome_modelo": "KCA", "respostas_corretas": respostas_corretas_kca_arc_c},
            {"nome_modelo": "ICD", "respostas_corretas": respostas_corretas_icd_arc_c},
            {"nome_modelo": "WikiChat", "respostas_corretas": respostas_corretas_wikichat_arc_c},
            {"nome_modelo": "RepE", "respostas_corretas": respostas_corretas_repe_arc_c},
        ]
    },
    {
        "nome_benchmark": "OAB", "total_questoes": 2210,
        "respostas_base": respostas_corretas_base_oab,
        "modelos_comparacao_input": [
            {"nome_modelo": "KCA", "respostas_corretas": respostas_corretas_kca_oab},
            {"nome_modelo": "ICD", "respostas_corretas": respostas_corretas_icd_oab},
            {"nome_modelo": "WikiChat", "respostas_corretas": respostas_corretas_wikichat_oab},
            {"nome_modelo": "RepE", "respostas_corretas": respostas_corretas_repe_oab},
        ]
    }
]

In [5]:
lista_resultados_finais = []
print("--- Iniciando Análise Comparativa de Acurácia ---")

for benchmark_data in dados_benchmarks_input:
    nome_b = benchmark_data["nome_benchmark"]
    N_B = benchmark_data["total_questoes"]
    respostas_B = benchmark_data["respostas_base"]
    print(f"\n\n============================================================")
    print(f"Processando Benchmark: {nome_b} (Total de Questões: {N_B})")
    print(f"============================================================")
    print(f"Modelo Base (Llama-2 7b): {respostas_B} respostas corretas de {N_B}.")

    for modelo_comp_data in benchmark_data["modelos_comparacao_input"]:
        nome_M = modelo_comp_data["nome_modelo"]
        respostas_M = modelo_comp_data["respostas_corretas"]
        N_M = N_B
        print(f"\n--- Comparando Llama-2 7b com: {nome_M} ---")
        print(f"  {nome_M}: {respostas_M} respostas corretas de {N_M}.")
        resultados_estatisticos = calcular_estatisticas_comparacao_acuracia(respostas_M, N_M, respostas_B, N_B)
        print(f"  Acurácia {nome_M} (p_M): {resultados_estatisticos['acuracia_M']:.4f}")
        print(f"  Acurácia Llama-2 7b (p_B): {resultados_estatisticos['acuracia_B']:.4f}")
        print(f"  Diferença nas Acurácias (p_M - p_B): {resultados_estatisticos['diferenca_acuracias']:.4f}")
        print(f"  Erro Padrão da Diferença: {resultados_estatisticos['erro_padrao_diferenca']:.4f}")
        print(f"  IC 95% para (p_M - p_B): [{resultados_estatisticos['ic_inferior']:.4f}, {resultados_estatisticos['ic_superior']:.4f}]")
        print(f"  Estatisticamente Significante (p < 0.05): {resultados_estatisticos['estatisticamente_significativo']}")
        print(f"  Interpretação: {resultados_estatisticos['interpretacao']}")
        lista_resultados_finais.append({
            "Benchmark": nome_b, "Modelo Base": "Llama-2 7b",
            "Acurácia Base (p_B)": resultados_estatisticos['acuracia_B'], "N Questões": N_B,
            "Modelo Comparado": nome_M, "Acurácia Comparado (p_M)": resultados_estatisticos['acuracia_M'],
            "Diferença (p_M - p_B)": resultados_estatisticos['diferenca_acuracias'],
            "IC 95% Inferior": resultados_estatisticos['ic_inferior'], "IC 95% Superior": resultados_estatisticos['ic_superior'],
            "Significativo (p < 0.05)": resultados_estatisticos['estatisticamente_significativo'],
            "Interpretação Detalhada": resultados_estatisticos['interpretacao']
        })

--- Iniciando Análise Comparativa de Acurácia ---


🔎 Processando Benchmark: TruthfulQA mc1 (Total de Questões: 817)
Modelo Base (Llama-2 7b): 120 respostas corretas de 817.

--- Comparando Llama-2 7b com: KCA ---
  KCA: 319 respostas corretas de 817.
  Acurácia KCA (p_M): 0.3905
  Acurácia Llama-2 7b (p_B): 0.1469
  Diferença nas Acurácias (p_M - p_B): 0.2436
  Erro Padrão da Diferença: 0.0211
  IC 95% para (p_M - p_B): [0.2022, 0.2849]
  Estatisticamente Significante (p < 0.05): True
  Interpretação: A acurácia do modelo comparado foi estatisticamente SUPERIOR. O IC de 95% para a diferença (p_M - p_B) é [0.2022, 0.2849]. (Não contém zero).

--- Comparando Llama-2 7b com: ICD ---
  ICD: 145 respostas corretas de 817.
  Acurácia ICD (p_M): 0.1775
  Acurácia Llama-2 7b (p_B): 0.1469
  Diferença nas Acurácias (p_M - p_B): 0.0306
  Erro Padrão da Diferença: 0.0182
  IC 95% para (p_M - p_B): [-0.0051, 0.0663]
  Estatisticamente Significante (p < 0.05): False
  Interpretação: A acurácia do 

In [6]:
#@markdown ---
#@markdown ### ⚠️ Nota para Abrir o CSV Corretamente:
#@markdown Se o arquivo CSV parecer desorganizado ao abrir no Excel (ou similar):
#@markdown 1. Tente usar a opção **`Dados`** -> **`De Texto/CSV`** (ou **`Importar Dados de Texto`**).
#@markdown 2. Na janela de importação, verifique de que o **`Delimitador`** está configurado para **`Vírgula (,)`**.
#@markdown 3. Verifique se a **`Origem do Arquivo`** ou **`Codificação`** está configurada para **`65001: Unicode (UTF-8)`**.

if lista_resultados_finais:
    print("\n\n--- Preparando CSV com Resultados de Todas as Comparações ---")
    df_resultados = pd.DataFrame(lista_resultados_finais)

    # Não é necessário formatar os números como strings antes de salvar no CSV,
    # pois o Excel e outros softwares de planilhas lidam melhor com dados numéricos.
    # A formatação para 4 casas decimais é preservada porque os valores já são floats.

    nome_arquivo_csv = 'resultados_comparacao_acuracia_qa.csv'
    df_resultados.to_csv(
        nome_arquivo_csv,
        index=False,
        encoding='utf-8-sig', # Boa compatibilidade com Excel para caracteres especiais
        sep=',',
        quoting=csv.QUOTE_NONNUMERIC # Cita campos não numéricos
    )
    print(f"\nOs resultados foram salvos em '{nome_arquivo_csv}'.")
    print("Iniciando download do arquivo CSV...")
    files.download(nome_arquivo_csv)
else:
    print("\nNenhum resultado foi gerado para salvar em CSV.")

print("\n--- Fim do Script ---")



--- 💾 Preparando CSV com Resultados de Todas as Comparações ---

Os resultados foram salvos em 'resultados_comparacao_acuracia_qa.csv'.
Iniciando download do arquivo CSV...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Se o arquivo CSV parecer desorganizado ao abrir no Excel (ou similar):
1. Tente usar a opção 'Dados' -> 'De Texto/CSV' (ou 'Importar Dados de Texto').
2. Na janela de importação, certifique-se de que o 'Delimitador' está configurado para 'Vírgula'.
3. Verifique se a 'Origem do Arquivo' ou 'Codificação' está configurada para '65001: Unicode (UTF-8)'.

--- Fim do Script ---
